In [127]:
from scipy.optimize import broyden2 as solve
import numpy as np

In [128]:
# parameters
mass = 1
sprung_inertia = np.array([[1, 1, 1], [1, 1, 1], [1, 1, 1]])
unsprung_inertia = np.array([[1, 1, 1], [1, 1, 1], [1, 1, 1]])

# sweep
rotational_velocities = np.array([0, 0, 1])
translation_velocities = np.array([1, 1, 0])

In [129]:
def car_loads(translation_accelerations, rotational_accelerations, roll, pitch, heave):
    # aero loads is roll/heave/pitch dependent
    # tire loads is aero force dependent
    # force/moment is aero & tire load dependent
    return translation_accelerations

In [131]:
def DOF6_motion_residuals(x, body_slip, steered_angle, velocity, yaw_rate):
    x_double_dot, y_double_dot, yaw_acceleration, roll, pitch, heave = x
    translation_accelerations = [x_double_dot, y_double_dot, 0]
    rotational_accelerations = [0, 0, yaw_acceleration]

    forces, moments = car_loads(translation_accelerations, rotational_accelerations, roll, pitch, heave)

    residuals_translation = mass * (translation_accelerations + np.cross(rotational_velocities, translation_velocities)) - \
        forces

    residuals_rotation = sprung_inertia.dot(rotational_accelerations) + np.cross(rotational_velocities, sprung_inertia.dot(rotational_velocities)) -\
        + unsprung_inertia.dot(rotational_accelerations) - moments
    return np.array([*residuals_translation, *residuals_rotation])


In [132]:
translation_accelerations, rotational_accelerations = [1, 1, 1], [1, 1, 1]
x = [translation_accelerations, rotational_accelerations]

for x in steered_angles:
    
    specific_residual_func = lambda x: DOF6_motion_residuals(x, body_slip, steered_angle, velocity, yaw_rate)
    solve(specific_residual_func, x, verbose=True)

0:  |F(x)| = 1.07418; step 1
1:  |F(x)| = 1; step 1
2:  |F(x)| = 1; step 1
3:  |F(x)| = 1; step 1
4:  |F(x)| = 1; step 1
5:  |F(x)| = 1; step 1
6:  |F(x)| = 1; step 1
7:  |F(x)| = 1; step 1
8:  |F(x)| = 1; step 1
9:  |F(x)| = 0.195729; step 1
10:  |F(x)| = 0.0503342; step 1
11:  |F(x)| = 0.00572985; step 1
12:  |F(x)| = 4.1943e-05; step 1
13:  |F(x)| = 1.44312e-07; step 1


array([[ 2.69588771e+16, -2.69588771e+16,  1.00000000e+00],
       [-1.00000014e+00,  9.99999947e-01, -1.06241901e-07]])

In [ ]:
translation_accelerations, rotational_accelerations = [1, 1, 1], [1, 1, 1]
x = [translation_accelerations, rotational_accelerations]

suspension = Suspension()
aero = Aero()
vehicle = Vehicle(suspension, aero)


for y in body_slips:
    for x in steered_angles:
        vehicle.state.body_slip = y
        vehicle.state.steered_angle = x

        specific_residual_func = lambda x: DOF6_motion_residuals(x, vehicle)
        solve(specific_residual_func, x, verbose=True)